# Программная реализация машиного обучения для определения биомаркеров болезни Паркинсона по ЭЭГ

Используемый датасет: https://bit.ly/3pP1pts

(dataset is a public dataset (Anjum et al., 2020) recorded by the University of Iowa (UI; Iowa City, Iowa))

In [3]:
%pip install mne scipy numpy matplotlib numba scikit-learn mat73 pandas --quiet

Note: you may need to restart the kernel to use updated packages.


## Data loading

In [4]:
import mne
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from mne.decoding import CSP
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit, KFold
from sklearn.model_selection import cross_val_score
import mat73 # поддержка загрузки функций матлаба 7.3

In [5]:
# используемые версии
mne.sys_info()

Platform             macOS-14.6.1-arm64-arm-64bit
Python               3.9.19 (main, May  6 2024, 14:39:30)  [Clang 14.0.6 ]
Executable           /Users/evakhromeeva/Projects/BeSMPSegmentation/.conda/bin/python
CPU                  arm (8 cores)
Memory               16.0 GB

Core
├☑ mne               1.8.0 (latest release)
├☑ numpy             1.26.4 (OpenBLAS 0.3.23.dev with 8 threads)
├☑ scipy             1.10.1
└☑ matplotlib        3.8.4 (backend=module://matplotlib_inline.backend_inline)

Numerical (optional)
├☑ sklearn           1.1.3
├☑ numba             0.60.0
├☑ pandas            2.0.3
├☑ h5py              3.12.1
└☐ unavailable       nibabel, nilearn, dipy, openmeeg, cupy, h5io

Visualization (optional)
└☐ unavailable       pyvista, pyvistaqt, vtk, qtpy, ipympl, pyqtgraph, mne-qt-browser, ipywidgets, trame_client, trame_server, trame_vtk, trame_vuetify

Ecosystem (optional)
├☑ mne-bids          0.15.0
└☐ unavailable       mne-nirs, mne-features, mne-connectivity, mne-icalabel, 

In [6]:
data_dath_control = 'data/raw/IowaDataset/Control/' # контрольная группа
data_dath_PD = 'data/raw/IowaDataset/PD/' # с болезнью Паркинсона
data_file = './data/raw/IowaData.mat' # склеенный большой датасет в формате MathLab

In [7]:
load_path = data_file

In [8]:
file = data_file #data_dath_PD +'PD1001.mat'

load_data = mat73.loadmat(file) # загрузка для матлаба 7.3
#load_data = sio.loadmat(file) # то же самое для другого матлаба

In [22]:
print(len(load_data['EEG'])) # в EEG лежат все данные
#print(load_data['Filename'])
print(load_data['Channel_location']) # каналы
print(len(load_data['Channel_location'])) # каналы
print(load_data['Filenames']) # Файлы, которые попали сюда (и PD, и Control)
print(len(load_data['Filenames'][0])) # Файлы PD
print(len(load_data['Filenames'][0])) # Файлы Control
#load_data


64
['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz', 'Fp1']
64
[[['PD1001'], ['PD1021'], ['PD1031'], ['PD1061'], ['PD1091'], ['PD1101'], ['PD1151'], ['PD1201'], ['PD1251'], ['PD1261'], ['PD1311'], ['PD1571'], ['PD1661'], ['PD1681']], [['Control1021'], ['Control1041'], ['Control1061'], ['Control1081'], ['Control1101'], ['Control1111'], ['Control1191'], ['Control1201'], ['Control1211'], ['Control1231'], ['Control1291'], ['Control1351'], ['Control1381'], ['Control1411']]]
14
14


In [ ]:
df1 = pd.DataFrame(data=load_data['EEG'][0][0])
df2 = pd.DataFrame(data=load_data['EEG'][0][1])
df3 = pd.DataFrame(data=load_data['EEG'][1][0])
df4 = pd.DataFrame(data=load_data['EEG'][1][1])


In [24]:
df1

,0
0,None
1,None
2,None
3,None
4,None
5,None
6,None
7,None
8,None
9,None


In [33]:
df2

,0,1,2,3,4,5,6,7,8,9,...,164500,164501,164502,164503,164504,164505,164506,164507,164508,164509
0,6.903133,6.604849,7.185431,6.057107,-0.392409,-4.252577,-2.301932,-4.306355,-6.554317,-2.650171,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-77.315666,-70.946869,-127.705673,-139.972321,-126.164970,-95.029991,-24.895681,-25.864864,-109.390419,-153.005432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-40.582520,-41.511360,-49.763115,-45.514305,-34.382912,-32.335403,-33.510677,-34.783257,-35.469620,-36.888103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,511.677338,506.454590,503.234985,518.861267,535.166626,531.108887,515.823364,517.531921,526.612000,528.805725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36.690037,39.727100,44.472046,52.730770,51.906422,48.445950,49.722118,49.875034,47.147453,49.693333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,118.060478,113.876762,125.561104,127.233795,121.241081,133.655640,136.839493,119.615173,111.964668,109.491379,...,-121.876175,-110.224098,-101.797493,-102.014618,-102.134041,-102.595169,-107.059509,-116.014122,-117.594208,-112.387695
6,11.960566,16.254471,16.153254,10.632912,7.164621,8.237681,7.113484,6.624280,10.285016,10.136663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,36.091618,36.329723,49.407734,45.345806,39.722744,39.471603,30.335073,34.578136,48.291306,49.600014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5.460632,3.606283,-7.767674,-16.990282,-25.233858,-20.830410,-13.498936,-14.713244,-7.774068,-2.350312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5.347610,10.580648,12.394724,2.881768,-0.965240,2.560842,7.355277,14.540595,14.351879,8.304673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df3

,0,1,2,3,4,5,6,7,8,9,...,171340,171341,171342,171343,171344,171345,171346,171347,171348,171349
0,11.369680,7.434574,4.330541,8.843492,14.624516,22.991430,16.660795,1.300030,4.495663,5.249332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-16.478432,-22.823175,-29.556694,-31.894402,-32.864452,-34.566544,-38.074539,-40.312187,-42.744488,-44.931992,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.896951,4.358635,2.746406,3.868619,9.872634,12.213361,8.743703,6.788484,11.034109,10.786983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,58.718227,62.470016,63.828506,62.647907,65.812721,71.710625,71.894791,71.883598,76.901001,80.257057,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-49.209145,-50.227844,-48.561089,-48.115334,-47.962627,-49.225780,-52.002029,-54.045174,-55.062492,-48.902805,...,-17.034248,-13.082373,-13.13495,-16.946745,-18.26758,-18.074585,-15.538225,-13.051403,-9.686503,-10.765284
5,115.945175,115.697983,114.230339,116.473511,120.083031,123.935638,118.413483,111.477066,109.766472,113.084938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-4.742566,-6.305727,-7.624340,-7.575586,-6.306316,-5.183868,-5.184625,-6.405911,-5.674004,-3.038277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-24.076794,-23.473883,-25.556307,-30.713877,-34.405350,-30.333206,-26.457499,-28.002039,-27.153904,-27.526537,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-0.453070,5.888137,2.560953,-0.570005,-6.189959,-15.225806,-10.100952,0.734534,7.221869,10.533991,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-11.478978,-14.243970,-19.449238,-25.824741,-25.265833,-16.651583,-12.726603,-7.728667,-5.026740,-6.817126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df4

,0,1,2,3,4,5,6,7,8,9,...,164500,164501,164502,164503,164504,164505,164506,164507,164508,164509
0,-16.573851,-18.390152,-18.887766,-19.434061,-23.153576,-26.432650,-26.537416,-26.593332,-28.211508,-28.413412,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,82.487221,62.108253,36.559998,32.597935,-0.997471,-9.586926,43.438396,65.154343,23.640686,4.059491,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-14.218865,-21.153387,-24.473177,-21.494093,-15.049545,-11.096701,-13.540550,-15.544317,-15.497026,-15.547393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,153.793121,155.059250,158.131393,160.665649,159.635208,153.088272,147.080215,150.935730,153.813705,154.005722,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-13.905575,-12.677874,-9.741776,-5.439514,-4.018920,-4.990966,-4.742207,-1.368962,2.589184,5.179217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,70.173103,67.499184,69.610497,71.379456,66.020020,62.810226,59.894268,51.706444,52.211102,56.230747,...,14.531931,15.746505,18.523014,20.371134,23.292797,23.723753,21.469419,17.214054,14.619811,17.884846
6,-6.445714,-14.062609,-19.382753,-17.329391,-17.424742,-21.084011,-20.641268,-19.808075,-19.268047,-19.362844,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3.919180,7.397347,15.659005,17.376289,18.263124,16.952755,7.489615,4.375906,13.712484,20.654181,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6.608749,6.221406,4.857748,5.593788,5.646122,4.770838,5.799829,3.850435,4.049725,8.203475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-2.512596,-3.347239,-2.082335,-3.649418,-3.458494,-4.585797,-4.638714,-1.908808,0.771504,3.451059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Выглядит как будто у нас 28 каналов, которые мы зачем-то поделили по полушариям (те самыо 0 и 1 - понять бы, откуда нумерация и понадобится ли мне это). Так же есть много столбцов, в которых в основном NaN - их нужно удалить. По OX, кажется время. Нужно понять, в чет оно исчисляется (милисекунды/секунды или еще что-то).
Другие 0 и 1, очевидно, означают наличие или отсутствие подтвержденного диагноза болезни Паркинсона.

# Выборка train и test
Для подготовки выборки мне нужно склеить датасет по частотам и по наличию/осутствию. Притом сделать это нужно так, чтобы было возможно его перемешать и поделить на train и test.
! Поссмотреть, как это делалось в моем пробном колабе с mne!!!

In [29]:
eeg_data = np.array(load_data["x_train"]).T
label = np.array(load_data["y_train"])
print(eeg_data.shape)

KeyError: 'x_train'